In [1]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('Datasets//movies_metadata.csv')

C:\Users\Nihar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df.fillna('', inplace=True)

In [5]:
columns_needed = ['genres', 'overview', 'title']
df_modified = df[columns_needed]
df_modified

,genres,overview,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,Father of the Bride Part II
...,...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.,Subdue
45462,"[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...,Century of Birthing
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","When one of her hits goes wrong, a professiona...",Betrayal
45464,[],"In a small town live two brothers, one a minis...",Satan Triumphant


In [6]:
df_modified['genres'] = df_modified['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

C:\Users\Nihar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
df_modified.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    45466 non-null  object
 1   overview  45466 non-null  object
 2   title     45466 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [8]:
def combine_genres(col):
    genres = ''
    for i in range(len(col)):
        genres = genres + ' ' + col[i]
    return genres

In [9]:
df_modified['combined_genres'] = df_modified['genres'].apply(combine_genres)

C:\Users\Nihar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_modified.drop('genres', axis=1, inplace=True)

C:\Users\Nihar\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
df_modified

,overview,title,combined_genres
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation Comedy Family
1,When siblings Judy and Peter discover an encha...,Jumanji,Adventure Fantasy Family
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,Romance Comedy
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,Comedy Drama Romance
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,Comedy
...,...,...,...
45461,Rising and falling between a man and woman.,Subdue,Drama Family
45462,An artist struggles to finish his work while a...,Century of Birthing,Drama
45463,"When one of her hits goes wrong, a professiona...",Betrayal,Action Drama Thriller
45464,"In a small town live two brothers, one a minis...",Satan Triumphant,


In [12]:
df_final = df_modified.copy()

In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   overview         45466 non-null  object
 1   title            45466 non-null  object
 2   combined_genres  45466 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [14]:
def combine_frames(row):
    comb=''
    comb = row['overview'] + row['combined_genres']
    #comb = row['title'] + ' ' + row['overview'] + ' ' + row['tagline'] + ' ' + row['combined_genres']
    df_final['combined_data'] = comb

In [15]:
combine_frames(df_final)
#df_modified['combined_data'] = df_modified.apply(combine_frames)

In [16]:
df_final

,overview,title,combined_genres,combined_data
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...
...,...,...,...,...
45461,Rising and falling between a man and woman.,Subdue,Drama Family,Rising and falling between a man and woman. Dr...
45462,An artist struggles to finish his work while a...,Century of Birthing,Drama,An artist struggles to finish his work while a...
45463,"When one of her hits goes wrong, a professiona...",Betrayal,Action Drama Thriller,"When one of her hits goes wrong, a professiona..."
45464,"In a small town live two brothers, one a minis...",Satan Triumphant,,"In a small town live two brothers, one a minis..."


The next Cell used tfid instead of count vectorizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 1), min_df=0.0001, stop_words='english')

tfidf_matrix = tfidf.fit_transform(df_final["combined_data"])
tfidf_matrix.toarray()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index= df_final.index.tolist())

In [23]:
tfidf_df.shape

(45466, 20748)

In [25]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)

latent_matrix = svd.fit_transform(tfidf_df)

latent_matrix.shape

(45466, 100)

In [26]:
explained = svd.explained_variance_ratio_.cumsum()

plt.plot(explained, '._', ms=16, color='red')
plt.xlabel('Singular value components')
plt.ylabel('Cumulative percent of variance')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
n =100

latent_matrix_df = pd.DataFrame(latent_matrix)

Decreasing size of count matrix

In [51]:
def get_title_from_index(index):
    return df_final[df_final.index==index]['title'].values[0]
def get_index_from_title(title):
    return df_final[df_final.title==title].index[0]

In [67]:
movie_user_likes = "3 Idiots"

In [68]:
movie_index = get_index_from_title(movie_user_likes)
movie_index

14732

In [69]:
similar_movies = list(enumerate(combined[movie_index]))

In [70]:
sorted_similar_movies = sorted(similar_movies, key = lambda x:x[1], reverse = True)[1:]

In [71]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>10:
        break

Top 5 similar movies to 3 Idiots are:

Wuthering Heights
Whispering Corridors
The Woods
Chase a Crooked Shadow
From Dusk Till Dawn
City By The Sea
Grand Prix
Around the World in 80 Days
What's Up, Doc?
Fat Man and Little Boy
Space Cowboys


In [45]:
df_final[df_final.index==1773]

,title,overview,tagline,combined_genres,combined_data
1773,Godzilla,When a freighter is viciously attacked in the ...,Size does matter.,Science Fiction Action Thriller,Godzilla When a freighter is viciously attacke...


In [188]:
df_final[df_final.title=="Two Evil Eyes"]

,title,overview,tagline,combined_genres,combined_data
6259,Two Evil Eyes,Two horror segments based on Edgar Allan Poe s...,When I wake you... you'll be dead,Horror Thriller,Two Evil Eyes Two horror segments based on Edg...


In [72]:
np.save('Cosine_matrix1.csv', combined)